In [ ]:
import pandas as pd
import numpy as np
import os
import gc
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
from skimage.io import imread, imshow
from skimage.transform import resize
from keras.models import Model, load_model
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import time
import tensorflow as tf

# Don't Show Warning Messages
import warnings
warnings.filterwarnings('ignore')
from patchify import patchify, unpatchify
import random
 
%matplotlib inline
import glob
from pathlib import Path
import math

seed = 42
np.random.seed = seed
from keras.utils.np_utils import normalize
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras import backend as K
from numpy import asarray

from keras.utils import normalize
from PIL import Image


In [ ]:
base_ptch_512.model.load_weights('model_100%_V1.h5')

In [ ]:


import numpy as np

def f1_score(y_true, y_pred):
    """Calculate the F1 score given the true and predicted labels."""
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    if tp + fp == 0 or tp + fn == 0:
        return 0
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if precision + recall == 0:
        return 0
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

def accuracy(y_true, y_pred):
    """Calculate the accuracy given the true and predicted labels."""
    num_correct = np.sum(y_true == y_pred)
    num_total = y_true.size
    acc = num_correct / num_total
    return acc

def recall(y_true, y_pred):
    """Calculate the recall given the true and predicted labels."""
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    if tp + fn == 0:
        return 0
    recall = tp / (tp + fn)
    return recall

def precision(y_true, y_pred):
    """Calculate the precision given the true and predicted labels."""
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    if tp + fp == 0:
        return 0
    precision = tp / (tp + fp)
    return precision

def jaccard_similarity(y_true, y_pred):
    """Calculate the Jaccard similarity given the true and predicted labels."""
    intersection = np.sum((y_true == 1) & (y_pred == 1))
    union = np.sum((y_true == 1) | (y_pred == 1))
    jaccard = (intersection + 1) / (union + 1)
    return jaccard


In [ ]:
def Test_(model, test_list, test_gen):
   
    label_sets = []
    iou = []
    f1=[]
    acc=[]
    recl =[]
    prec=[]
    for i, batch in enumerate(test_gen):
        labels = batch[1]
        test_img = batch[0]
        


        test_preds = model.predict(test_img, verbose=0)
        preds_test_thresh = (test_preds >= 0.35).astype(np.uint8)
        
        jcard_sim = jaccard_similarity( labels,preds_test_thresh)
        iou.append(jcard_sim)
        f1.append(f1_score( labels,preds_test_thresh))
        acc.append(accuracy( labels,preds_test_thresh))
        recl.append(recall( labels,preds_test_thresh))
        prec.append(precision( labels,preds_test_thresh))
        

        if i > len(test_list)/32:
           
            break
    average_similarity = np.mean(iou)
    average_acc = np.mean(acc)
    average_f1 = np.mean(f1)
    average_recall = np.mean(recl)
    average_precl = np.mean(prec)

    return average_similarity, average_acc, average_f1, average_recall, average_precl



In [ ]:
def pad_image(image, patch_size):
    x_size, y_size = get_valid_sz(image.shape)
    height, width = image.shape
    padded_image = np.pad(image, ((0, x_size - height), (0, y_size - width)), mode='constant')
    return padded_image

kernel_ = np.ones((10, 10), np.uint8)

def full_image_inference1(model, data_path, lab_path, img_pth, display=False, label_inc=True):
    image = cv2.imread(data_path + img_pth, 0)
    aspect_ratio = image.shape[1] / image.shape[0]

    new_size = 2400
    if aspect_ratio > 1:
        new_width = new_size
        new_height = int(new_width / aspect_ratio)
    else:
        new_height = new_size
        new_width = int(new_height * aspect_ratio)
    image = cv2.resize(image, (new_width, new_height))

    # pad image to make sure its size is a multiple of the patch size
    padded_image = pad_image(image, IMG_HEIGHT)

    original_shape = image.shape

    if label_inc:
        label_img_ = cv2.imread(lab_path + "M" + img_pth, 0)
        label_img = cv2.resize(label_img_, (new_width, new_height))/255.

    patches = patchify(padded_image, (IMG_HEIGHT, IMG_WIDTH), step=step_size)

    predicted_patches = []

    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            single_patch = patches[i, j, :, :] / 255.0
            single_patch = (single_patch - mean) / std
            single_patch_input = single_patch.reshape(1, IMG_WIDTH, IMG_HEIGHT, 1)
            single_patch_prediction = model.predict(single_patch_input, verbose=0)
            predicted_patches.append(single_patch_prediction)

    predicted_patches = np.array(predicted_patches)
    predicted_patches_reshaped = np.reshape(predicted_patches, (patches.shape[0], patches.shape[1], IMG_HEIGHT, IMG_WIDTH))

    reconstructed_image = unpatchify(predicted_patches_reshaped, (padded_image.shape[0], padded_image.shape[1]))

    # remove padding
    reconstructed_image = reconstructed_image[:original_shape[0], :original_shape[1]]

    reconstructed_image = (reconstructed_image > 0.5).astype(np.uint8)
    
  

    overlay_image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    overlay_image[label_img > 0.5] = (0, 255, 0)  # Green color for true mask
    overlay_image[reconstructed_image > 0.5] = (255, 0, 0)  # Red color for predicted mask

    if display:
        fig, axes = plt.subplots(ncols=4, figsize=(12, 10))

        # Display the images on the axes
        axes[0].imshow(image, cmap='gray')
        axes[0].set_title('Full-sized Image')

        axes[1].imshow(label_img, cmap='gray')
        axes[1].set_title('Label')

        axes[2].imshow(reconstructed_image, cmap='gray')
        axes[2].set_title('Prediction')

        axes[3].imshow(overlay_image)
        axes[3].set_title('Overlay')

    else:
        if label_inc:
            return label_img.reshape(1, label_img.shape[0], label_img.shape[1], 1), reconstructed_image.reshape(1, reconstructed_image.shape[0], reconstructed_image.shape[1], 1)
        else:
            return reconstructed_image.reshape(reconstructed_image.shape[0], reconstructed_image.shape[1])


In [ ]:
data_path = r'F:\semi_\Final\Bi_test_images\\'   
lab_path = r'F:\semi_\Final\Bi_test_masks\\'
test_data_list = os.listdir(data_path)
ind_ = random.randint(0, len(test_data_list)-1)
print(ind_)
full_image_inference1(base_ptch_512.model,data_path,lab_path , test_data_list[ind_], True)


In [ ]:
#data_path = r'E:\\testimage1111\\'   
#lab_path = r'E:\testmask.1111\\'

def Large_image_metric(model, p_list, data_path, lab_path):
    iou=[]
    acc =[]
    f1=[]
    recl =[]
    prec = []
    c = 0
    for file in p_list:
        labels, preds_test_thresh = full_image_inference1(model,data_path, lab_path, file)
        labels = (labels >= 0.5)
        preds_test_thresh = (preds_test_thresh >= 0.5)
       
        jcard_sim = jaccard_similarity( labels,preds_test_thresh)
        iou.append(jcard_sim)
        f1.append(f1_score( labels,preds_test_thresh))
        acc.append(accuracy( labels,preds_test_thresh))
        recl.append(recall( labels,preds_test_thresh))
        prec.append(precision( labels,preds_test_thresh))
       
     
        c+=1
#         if jcard_sim <0.3:
#             print(c, jcard_sim, file)
       

       
        
           
    average_similarity = np.mean(iou)
    average_acc = np.mean(acc)
    average_f1 = np.mean(f1)
    average_recall = np.mean(recl)
    average_precl = np.mean(prec)


    print(f"Jaccard sim :: {round(average_similarity,3)} , Accuracy :: {round(average_acc,3)},  Precision :: {round(average_precl,3)}, f1 :: {round(average_f1,3)}, Recall :: {round(average_recall,3)} ")


In [ ]:
include_full_size_img= False

test_data_list = os.listdir(data_path)
Large_image_metric(base_ptch_512.model, test_data_list, data_path, lab_path)